## Общая предобработка данных

In [1]:
import os

import pandas as pd
import numpy as np
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
)


DATASET_URL = "https://storage.yandexcloud.net/cbr-press-release-classifier/cbr-press-releases.csv"

Загрузим датасет пресс-релизов, полученных с сайта ЦБ РФ:

In [2]:
df = pd.read_csv(DATASET_URL, parse_dates=["date"])
df.tail()

,date,link,title,release,inflation,rate,usd,usd_cur_change_relative,target_categorial,target_absolute,target_relative
96,2024-10-25,http://www.cbr.ru/press/pr/?file=25102024_1330...,Банк России принял решение повысить ключевую с...,Совет директоров Банка России 25 октября 2024 ...,8.63,21.0,96.7402,1.061800,0.0,0.0,1.0
97,2024-12-20,http://www.cbr.ru/press/pr/?file=20122024_1330...,Банк России принял решение сохранить ключевую ...,Совет директоров Банка России 20 декабря 2024 ...,8.88,21.0,103.4207,1.069056,0.0,0.0,1.0
98,2025-02-14,http://www.cbr.ru/press/pr/?file=14022025_1330...,Банк России принял решение сохранить ключевую ...,Совет директоров Банка России 14 февраля 2025 ...,9.92,21.0,91.0313,0.880204,0.0,0.0,1.0
99,2025-03-21,http://www.cbr.ru/press/pr/?file=21032025_1330...,Банк России принял решение сохранить ключевую ...,Совет директоров Банка России 21 марта 2025 го...,10.06,21.0,84.3955,0.927104,0.0,0.0,1.0
100,2025-04-25,http://www.cbr.ru/press/pr/?file=25042025_1330...,Банк России принял решение сохранить ключевую ...,Совет директоров Банка России 25 апреля 2025 г...,NaN,NaN,82.8559,0.981757,NaN,NaN,NaN


`rate` - величина ключевой ставки, которую объявят на следующем заседании. Мы хотели бы знать эту величину. Но мы знаем текущую ставку и эту информацию можно добавить в наши данные сдвинув ставку на один.

In [3]:
df["rate"] = df.rate.shift(1)

В сентябре 2013 года ЦБ ввел ключевую ставку, и назначил ее величину в 5,5% заполним пропущенное значение.

In [4]:
df.loc[0, "rate"] = 5.5

Дату сделаем индексом, ссылку на пресс-релиз выбросим, эта переменная не поможет в предсказании таргета.

In [5]:
df.set_index("date", inplace=True)
df.drop("link", axis=1, inplace=True)

Для последнего релиза неизвестна целевая переменная (направление изменения ключевой ставки), поэтому исключим его из датасета и сохраним отдельно:

In [6]:
df.sort_values("date", inplace=True)
cur_pr = df.tail(1)
df = df[:-1]

Мы хотим предсказывать направление ставки рефинансирования `target_categorial`, выделим ее как целевую переменную. `target_absolute` и `target_relative` содержит в себе информацию о целевой переменной удалим их из матрицы признаков.

In [7]:
X = df.drop(["target_categorial", "target_absolute", "target_relative"], axis=1)
y = df["target_categorial"]

Сохраним X и y в папку data, чтобы пользоваться ими во всех экспериментах

In [8]:
directory = "../data"

if not os.path.exists(directory):
    os.makedirs(directory)

X.to_csv("../data/x.csv")
y.to_csv("../data/y.csv")

## Минимальный безлайн 

Направление изменения ставки будем предсказывать таким, как в последнем решении. До 13 сентября ключевой ставки не существовало, поэтому мы не можем предсказать таким способом для первого наблюдения, будем оценивать качество бейзлайна без него.

In [9]:
# вероятность 1 для соответствующего класса
def get_proba(y):
    proba = np.zeros((1, 3))
    proba[0, int(y + 1)] = 1.0
    return proba


y_pred = y.shift(1)
accuracy = accuracy_score(y[1:], y_pred[1:])
f1 = f1_score(y[1:], y_pred[1:], average="macro")
precision = precision_score(y[1:], y_pred[1:], average="macro")
recall = recall_score(y[1:], y_pred[1:], average="macro")
roc_auc_ovr = roc_auc_score(
    y[1:],
    np.concatenate(list(map(get_proba, y_pred[1:]))),
    average="macro",
    multi_class="ovr",
)
roc_auc_ovo = roc_auc_score(
    y[1:],
    np.concatenate(list(map(get_proba, y_pred[1:]))),
    average="macro",
    multi_class="ovo",
)
min_baseline = pd.DataFrame(
    {
        "accuracy": accuracy,
        "f1": f1,
        "precision": precision,
        "recall": recall,
        "roc_auc_ovr": roc_auc_ovr,
        "roc_auc_ovo": roc_auc_ovo,
    },
    index=["min_baseline"],
)

In [10]:
min_baseline

,accuracy,f1,precision,recall,roc_auc_ovr,roc_auc_ovo
min_baseline,0.666667,0.669899,0.669899,0.669899,0.746871,0.752424


Это будет нашим ориетиром в экспериментах. Будем хранить качество наших моделей в `metrics.csv`

In [11]:
min_baseline.to_csv("metrics.csv")